In [2]:
pip install torch

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   - -------------------------------------- 6.0/204.1 MB 32.3 MB/s eta 0:00:07
   -- ------------------------------------- 13.9/204.1 MB 34.2 MB/s eta 0:00:06
   ---- ----------------------------------- 21.2/204.1 MB 34.9 MB/s eta 0:00:06
   ----- ---------------------------------- 29.1/204.1 MB 36.0 MB/s eta 0:00:05
   ------- -------------------------------- 36.7/204.1 MB 36.4 MB/s eta 0:00:05
   -------- ------------------------------- 45.4/204.1 MB 36.9 MB/s eta 0:00:05
   ---------- ----------------------------- 53.0/204.1 MB 36.9 MB/s eta 0:00:05
   ----------- ---------------------------- 60.6/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install kagglehub

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------------- ------------------ 6.3/11.5 MB 36.1 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 32.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -------------------------- ------------- 8.4/12.6 MB 39.4 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 35.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
import kagglehub
import torch
import pandas as pd
import os
from pathlib import Path


In [39]:
path = os.path.dirname(os.path.realpath('__file__'))
car_pic = os.path.join(path, r'MRI_Tumor_Detection\TTV\Br35H-Mask-RCNN\annotations_all.json')

In [47]:
car_pic

'C:\\Users\\ijfre\\tester\\MRI_Tumor_Detection\\torch\\MRI_Tumor_Detection\\TTV\\Br35H-Mask-RCNN\\annotations_all.json'

In [25]:
dt = pd.read_json(r'MRI_Tumor_Detection\TTV\Br35H-Mask-RCNN\annotations_all.json')

FileNotFoundError: File MRI_Tumor_Detection\TTV\Br35H-Mask-RCNN\annotations_all.json does not exist

In [24]:
dt

,y0.jpg19127,y1.jpg59363,y2.jpg14060,y3.jpg19888,y4.jpg15178,y5.jpg13964,y6.jpg26744,y7.jpg16681,y8.jpg25031,y9.jpg8501,...,y791.jpg11822,y792.jpg16905,y793.jpg7103,y794.jpg12512,y795.jpg13172,y796.jpg46313,y797.jpg74904,y798.jpg9683,y799.jpg14604,y800.jpg19201
filename,y0.jpg,y1.jpg,y2.jpg,y3.jpg,y4.jpg,y5.jpg,y6.jpg,y7.jpg,y8.jpg,y9.jpg,...,y791.jpg,y792.jpg,y793.jpg,y794.jpg,y795.jpg,y796.jpg,y797.jpg,y798.jpg,y799.jpg,y800.jpg
size,19127,59363,14060,19888,15178,13964,26744,16681,25031,8501,...,11822,16905,7103,12512,13172,46313,74904,9683,14604,19201
regions,"[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all...",...,"[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'ellipse', 'cx'...","[{'shape_attributes': {'name': 'ellipse', 'cx'...","[{'shape_attributes': {'name': 'ellipse', 'cx'...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'ellipse', 'cx'...","[{'shape_attributes': {'name': 'ellipse', 'cx'...","[{'shape_attributes': {'name': 'ellipse', 'cx'...","[{'shape_attributes': {'name': 'polygon', 'all...","[{'shape_attributes': {'name': 'polygon', 'all..."
file_attributes,{},{},{},{},{},{},{},{},{},{},...,{},{},{},{},{},{},{},{},{},{}
